<a href="https://colab.research.google.com/github/tnfru/colab_research/blob/master/PricePred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
#import cupy as np
import time

## Read and transform data

In [0]:
data = pd.read_excel('https://drive.google.com/uc?export=download&id=1BTjcgqQO-NxpcIRyQu2_FNxUSBCvJt7c')

In [0]:
df = data.drop(['Kundenname'], axis=1)

In [0]:
def encode(df, column):
  df[column] = df[column].astype('category')
  df[column + "_cat"] = df[column].cat.codes

In [0]:
def encode_labels(df):
  labels_to_encode = ["Auftrags-ID", "Kunden-ID", "Bestelldatum", "Produkt-ID"]
  df = df.sort_values(by=["Bestelldatum"])

  for label in labels_to_encode:
    encode(df, label)

  labels_to_encode = ["Land/Region", "Bundesland", "Stadt"]
  df = df.sort_values(by=labels_to_encode)

  for label in labels_to_encode:
    encode(df, label)

  df = df.drop(["Auftrags-ID", "Kunden-ID", "Bestelldatum",
             "Land/Region", "Bundesland", "Stadt", "Versanddatum", 
             "Produkt-ID", "Zeilen-ID"], axis=1)
  return df

In [0]:
def get_separated_entities(df):
  df = encode_labels(df)

  items = df.drop(columns=["Kunden-ID_cat", "Bestelldatum_cat", "Auftrags-ID_cat", 
                "Land/Region_cat", "Bundesland_cat", "Stadt_cat", "Region", 
                "Segment", "Versandmodus"], axis=1)
  items = pd.get_dummies(items, columns=["Kategorie" , "Unterkategorie"])
  


  transactions = pd.get_dummies(df, columns=["Segment", "Versandmodus", "Region", "Kategorie"]).drop("Produktname", axis=1)

  customers = df.drop(columns=["Versandmodus", "Produktname", "Kategorie", "Gewinn",
                             "Bestelldatum_cat", "Produkt-ID_cat", "Kunden-ID_cat", "Unterkategorie"], axis=1)
  customers = pd.get_dummies(customers, columns=["Segment", "Region"])

  return items, customers, transactions
  

In [0]:
def cosine_sim(a, b):
  return a @ b / (np.linalg.norm(a) * np.linalg.norm(b))

In [0]:
def create_CPM(data):
  cpm_shape = data["Kunden-ID"].nunique(), data["Produkt-ID"].nunique()
  CPM = np.zeros(cpm_shape)
  cpm_inp = data.drop(columns=["Auftrags-ID", "Zeilen-ID", "Versanddatum", "Bestelldatum", 
                   "Versandmodus", "Kundenname", "Segment", "Stadt", "Bundesland", 
                   "Land/Region", "Region", "Kategorie", "Unterkategorie", "Umsatz",
                   "Menge", "Rabatt", "Gewinn", "Produktname"], axis=1)
  encode(cpm_inp, "Kunden-ID")
  encode(cpm_inp, "Produkt-ID")
  cpm_inp = cpm_inp.drop(["Kunden-ID", "Produkt-ID"], axis=1).values

  for i in range(len(cpm_inp)):
    cust_id = cpm_inp[i][0]
    prod_id = cpm_inp[i][1]

    CPM[cust_id][prod_id] = 1
  
  return CPM

In [0]:
def create_PPM(items, print_progress=False):
  ppm_shape = items["Produkt-ID_cat"].nunique(), items["Produkt-ID_cat"].nunique()
  PPM = np.zeros(ppm_shape)
  
  ppm_inp = items.drop(["Produktname", "Gewinn", "Rabatt"], axis=1)
  ppm_inp["Umsatz"] = ppm_inp["Umsatz"].values / ppm_inp["Menge"].values
  ppm_inp = ppm_inp.rename(columns={"Umsatz" : "Preis"}).drop(["Menge"], axis=1)
  ppm_inp['Preis'] = (ppm_inp['Preis'].values - ppm_inp['Preis'].values.mean()) / ppm_inp['Preis'].values.std()

  ppm_inp = ppm_inp.groupby(["Produkt-ID_cat"]).sum()
  number_of_entries = ppm_inp[["Kategorie_Bürobedarf", "Kategorie_Möbel", "Kategorie_Technik"]].values.sum(axis=1)
  ppm_inp = ppm_inp.divide(number_of_entries, axis=0)
  ppm_inp = ppm_inp.values

  n = ppm_shape[0]

  for i in range(n):
    for j in range(i + 1, n):
      PPM[i][j] = cosine_sim(ppm_inp[i], ppm_inp[j])

    if print_progress and i % 100 == 0:
      print(round(i / n, 2) * 100 , "% done")

  return PPM

In [0]:
def ppm_recommend(item_id,  n=1):
  return np.argsort(PPM[item_id])[-n:]

## Create CPM and set constants

In [0]:
def get_item_prior(epsilon=1e-3):
  #w'keit dass item von beliebigem nutzer gekauft wird
  return (CPM.sum(axis=0) + epsilon) / (num_users + 2 * epsilon) 

def get_user_prior(epsilon=1e-3):
  #w'keit dass nutzer ein beliebiges item kauft
  return (CPM.sum(axis=1) + epsilon) / (num_items + 2 * epsilon)

In [0]:
items, customers, transactions = get_separated_entities(df)

In [0]:
CPM = create_CPM(data)
#PPM = create_PPM(items)

num_users = CPM.shape[0]
num_items = CPM.shape[1]
item_prior = get_item_prior()
user_prior = get_user_prior()

## Naive Bayes Hybrid aproach

In [0]:
def user_based(user_id, item_id, epsilon=1e-3):
  k = CPM[user_id]
  K = np.tile(k, num_users).reshape(num_users, num_items)
  I = np.tile(CPM[:,item_id], num_items).reshape(num_items, num_users)

  cond = I * (CPM == K).T 
  items_probs = (cond.sum(axis=-1) + epsilon) / (CPM[:,item_id].sum() + 2 * epsilon)
  
  return (item_prior[item_id] * np.prod(items_probs)) ** (1/ (1 + num_items))

In [0]:
def item_based(user_id, item_id, epsilon=1e-3):
  k = CPM[:,item_id]
  K = np.tile(k, num_items).reshape(num_items, num_users).T
  U = np.tile(CPM[user_id], num_users).reshape(num_users, num_items)
  
  cond = U * (CPM == K)
  user_probs = (cond.sum(axis=-1) + epsilon) / (CPM[user_id].sum() + 2 * epsilon)

  return (user_prior[user_id] * np.prod(user_probs)) ** (1/ (1 + num_users))

In [0]:
def hybrid_score(user_id, item_id):
  return user_based(user_id, item_id) * item_based(user_id, item_id)

In [0]:
def recommend(user_id=None, item_id=None, n=4, print_time=True):
  recs = []

  if print_time:
    start = time.time()

  if user_id is None:
    for user_id in range(num_users):
      recs.append(hybrid_score(user_id, item_id))

  elif item_id is None:
    for item_id in range(num_items):
      recs.append(hybrid_score(user_id, item_id))

  else:
    return hybrid_score(user_id, item_id)

#  top_idx = np.argsort(recs)[-n:]
 # top_values = [recs[j] for j in top_idx]

  if print_time:
    print("This took", np.round(time.time() - start, 2), "seconds")

#  return top_idx, top_values
  return recs

In [0]:
foo = recommend(item_id=137)

AttributeError: ignored

In [0]:
bar = recommend(user_id=137)

In [0]:
item_recs = []

for item_id in range(num_items):
  item_recs.append(recommend(item_id=item_id, print_time=False))
  if item_id % 10 == 0:
    print(item_id / num_items, "%")